In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import scipy.sparse as sparse

In [3]:
import numpy as np
import pandas as pd

In [4]:
from thesislib.utils import pathutils

In [5]:
sparse_csv = pathutils.get_data_file("04_06_new_data/sparse/symptoms_04_16_4.csv")

In [6]:
df = pd.read_csv(sparse_csv, index_col='Index')

In [8]:
labels = df.LABEL
symptoms = df.SYMPTOMS
df = df.drop(columns=['LABEL', 'SYMPTOMS'])

In [12]:
dense_matrix = sparse.coo_matrix(df.values)

In [13]:
symptoms = symptoms.apply(lambda v: [int(idx) for idx in v.split(",")])

In [16]:
columns = []
rows = []
for idx, val in symptoms.iteritems():
    rows += [idx] * len(val)
    columns += val

In [19]:
data = np.ones(len(rows))

In [23]:
num_symptoms = 376

In [24]:
symptoms_coo = sparse.coo_matrix((data, (rows, columns)), shape=(df.shape[0], num_symptoms))

In [26]:
data_coo = sparse.hstack([dense_matrix, symptoms_coo])

In [30]:
data_csc = data_coo.tocsc()

In [29]:
from sklearn.model_selection import StratifiedShuffleSplit

In [31]:
label_values = labels.values

In [32]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(data_csc, label_values):
    train_data = data_csc[train_index]
    train_labels = label_values[train_index]
    test_data = data_csc[test_index]
    test_labels = label_values[test_index]

In [33]:
train_data.shape

(86336, 379)

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
clf = RandomForestClassifier(n_estimators=140, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=2, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [36]:
clf = clf.fit(train_data, train_labels)

In [37]:
from sklearn.metrics import accuracy_score, make_scorer

In [38]:
accuracy_scorer = make_scorer(accuracy_score)

In [39]:
train_score = accuracy_scorer(clf, train_data, train_labels)

In [41]:
test_score = accuracy_scorer(clf, test_data, test_labels)

In [42]:
print("Train Score: %.3f\n Test Score: %.3f" % (train_score, test_score))

Train Score: 0.946
 Test Score: 0.852
